<div style="color:red;font-weight:bold;background:yellow;text-align:center;padding:10px;border:solid">
    <h1>RUN IN EMR CLUSTER ONLY</h1>
    If the URL of the current page does not begin with "ec2", then do **NOT** proceed!
</div>

# SparkSQL
In this lab, we will look at using Spark with SQL to ingest and analyze datasets. 

### Connecting to PySpark

In [1]:
name = !hostname
if "dsa" in name[0]:
    raise RuntimeError("Only run this notebook in the EMR Cluster!")
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("pyspark-lab")
sc = SparkContext(conf=conf)

### Loading the Dataset

We will use Amazon Review data. 

**<span style="background:yellow">Please note, this will take a few minutes to stage into memory on the cluster.</span>**

In [3]:
from pyspark.sql import SQLContext

# To use Spark SQL we create a SQLContext from SparkContext
sqlContext = SQLContext(sc)

# Location of the dataset on HDFS
DATASET = '/datasets/amazon_reviews.json'

# Load a table with a JSON format reader and get the first 1000 rows
df = sqlContext.read.json(DATASET).limit(1000)

#### Let's look at the data

In [4]:
df.head()

Row(helpfulness_count=7, helpfulness_score=7, price=-1.0, product_id='B000179R3I', profile_name='Jeanmarie Kabala "JP Kabala"', score=4.0, summary='Periwinkle Dartmouth Blazer', text='I own the Austin Reed dartmouth blazer in every color in which they make it-- it is a staple of my business wardrobe. Well made, quality fabric, nicely tailored, classic lines, appropriate for a professional woman. (something that can be hard to find at times) It should be noted, however, that the periwinkle and raspberry colors are lovely, but the fabric and buttons are slightly different than the "classic" colors(lighter) and the linings and interfacings are not as substantial as the brown, navy, camel, red and ivory. It\'s still a good value, particularly as these are colors appropriate to warmer seasons and climates, but I was a bit surprised.', time=1182816000, title='Amazon.com', user_id='A3Q0VJTUO4EZ56')

## SparkSQL DataFrame vs. Pandas DataFrame
The DataFrame returned by SparkSQL is _NOT_ the same as a Pandas DataFrame. You can find the documentation what operations can be performed on a SparkSQL Dataframe [here](http://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#pyspark.sql.DataFrame).

To convert a SparkSQL DataFrame to a Pandas dataframe, you can do this:
```python
pandas_dataframe = spark_dataframe.toPandas()
```

## NoSQL

The SparkSQL DataFrame can be accessed similarly to a NoSQL database.
Note, these are similar to the capabilities of the Pandas dataframe.

**Viewing A Column**

In [5]:
df.select("score").show()

+-----+
|score|
+-----+
|  4.0|
|  5.0|
|  3.0|
|  4.0|
|  5.0|
|  5.0|
|  5.0|
|  5.0|
|  5.0|
|  5.0|
|  5.0|
|  5.0|
|  4.0|
|  5.0|
|  5.0|
|  5.0|
|  5.0|
|  5.0|
|  5.0|
|  5.0|
+-----+
only showing top 20 rows



**Filtering Data**
Note the chain of operations above.
If we decompose the chain into steps, we see that operations like filter and select give us new SparkSQL DataFrames.

In [6]:
df2 = df.filter(df["score"] < 3)

In [7]:
print(type(df2))

<class 'pyspark.sql.dataframe.DataFrame'>


In [8]:
df2.show()

+-----------------+-----------------+-----+----------+--------------------+-----+--------------------+--------------------+----------+--------------------+--------------+
|helpfulness_count|helpfulness_score|price|product_id|        profile_name|score|             summary|                text|      time|               title|       user_id|
+-----------------+-----------------+-----+----------+--------------------+-----+--------------------+--------------------+----------+--------------------+--------------+
|               11|                7|10.95|0595344550|         Book Reader|  1.0|            not good|I bought this boo...|1117065600|Whispers of the W...|A3Q12RK71N74LB|
|                2|                1|10.95|0595344550|LoveToRead "Actua...|  1.0|        Buyer beware|This is a self-pu...|1119225600|Whispers of the W...| AUR0VA5H0C66C|
|                0|                0|10.95|0595344550|        C. Robertson|  1.0|          The Worst!|A complete waste ...|1119916800|Whispers of


**Grouping Data**

In [9]:
df.groupBy("score").count().show()

+-----+-----+
|score|count|
+-----+-----+
|  4.0|  183|
|  5.0|  636|
|  3.0|   65|
|  1.0|   71|
|  2.0|   45|
+-----+-----+



## Using SQL with SparkSQL 

We can use SQL with Spark as well, which is very helpful for extracting information from the data we've found.


First, we must register the temporary table with PySpark

In [10]:
df.createOrReplaceTempView("review")

Now we can run any SQL query, and get a SparkSQL DataFrame back!

** What is the average price of products with a score of 4 or greater? **

In [11]:
query = "SELECT AVG(price) FROM review WHERE score >= 4"
sqlContext.sql(query).show()

+-----------------+
|       avg(price)|
+-----------------+
|6.041538461538451|
+-----------------+



** What is the overall average helpfulness score? **

In [12]:
query = "SELECT AVG(helpfulness_score) FROM review"
sqlContext.sql(query).show()

+----------------------+
|avg(helpfulness_score)|
+----------------------+
|                 4.973|
+----------------------+



** What is the average score of all products cheaper than $20? **

In [13]:
query = "SELECT AVG(score) FROM review WHERE price < 20"
sqlContext.sql(query).show()

+------------------+
|        avg(score)|
+------------------+
|4.2735527809307605|
+------------------+



** How many titles contain the phrase "Amazon"? **

In [14]:
query = "SELECT COUNT(*) FROM review WHERE title LIKE '%Amazon%'"
sqlContext.sql(query).show()

+--------+
|count(1)|
+--------+
|      16|
+--------+



### <span style="background:yellow">Your Turn</span> ###
Use SparkSQL to answer the following questions

**Q1: How many `text`s contain the phrases "nice" or "good"?**

In [15]:
query = "SELECT COUNT(*) FROM review WHERE (text LIKE '%good%' OR text LIKE '%nice%')"
sqlContext.sql(query).show()






+--------+
|count(1)|
+--------+
|     218|
+--------+



**Q2: What is the average price of products that have a score less than 3 and a helpfulness score less than 5?**

In [16]:
query = "SELECT AVG(price) FROM review WHERE ((score < 3) AND (helpfulness_score <5))"
sqlContext.sql(query).show()





+----------------+
|      avg(price)|
+----------------+
|7.79823529411765|
+----------------+



**Q3: How many products have product IDs beginning with the letter `B`?**

In [17]:
query = "SELECT product_id FROM review WHERE (product_id LIKE 'B%') LIMIT 5"
sqlContext.sql(query).show()




+----------+
|product_id|
+----------+
|B000179R3I|
|B000GKXY34|
|B000GKXY34|
|B00002066I|
|B00002066I|
+----------+



In [18]:
query = "SELECT COUNT(*) FROM review WHERE (product_id LIKE 'B%')"
sqlContext.sql(query).show()




+--------+
|count(1)|
+--------+
|     742|
+--------+



# Save your notebook, then `File > Close and Halt`

---